In [4]:
from aocd.models import Puzzle

In [8]:
puzzle = Puzzle(2019, 9)
puzzle_input = [int(x) for x in puzzle.input_data.split(",")]

In [9]:
# intcode computer from day 7

def get_opcode(value):
    # the opcode is the rightmost two digits of the first value in an instruction
    if len(str(value)) == 1: return value
    if len(str(value)) == 2: return int(str(value[1])) if str(value).startswith("0") else value
    else: return int(str(value)[len(str(value))-2:])
    

def determine_values_from_parameter_mode(parameter_mode, program, ip, ip_offset, relative_base_offset):
    parameter_mode = int(parameter_mode)
    
    if parameter_mode == 2: # relative mode
        return program[program[ip+ip_offset]+relative_base_offset]
    
    elif parameter_mode == 1: # immediate mode
        return program[ip+ip_offset]
    
    elif parameter_mode == 0: # position mode
        return program[program[ip+ip_offset]]
    
    else:
        raise ValueError("I can not understand the parameter mode for this value: {}".format(parameter_mode))
    
    
def intcode_computer(program, input_values, instruction_pointer_location):
    
    # map opcodes to parameter count
    opcode_parameter_map = dict()
    opcode_parameter_map["1"] = 3 # addition: summand1, summand2, adress of sum
    opcode_parameter_map["2"] = 3 # multiplication: factor1, factor2, adress of product
    opcode_parameter_map["3"] = 1 # input: where to store input address
    opcode_parameter_map["4"] = 1 # output: where to read output address
    opcode_parameter_map["5"] = 2 # jump-if-true: compare boolean of 2 parameters
    opcode_parameter_map["6"] = 2 # jump-if-false: compare boolean of 2 parameters
    opcode_parameter_map["7"] = 3 # less-than: compare values from 2 parameters, store 1 in position of third
    opcode_parameter_map["8"] = 3 # equals: compare values from 2 parameters, store 1 in position of third
    opcode_parameter_map["9"] = 1 # adjusts the relative base by the value of its only parameter
    
    output_value = None
        
    ip = instruction_pointer_location # instruction pointer
    relative_base_offset = 0
    
    # initialize program with a LOT more memory
    program.extend(0 for _ in range(100000))
    
    while True:
        # program[ip] is always the first value of the instruction. It contains the opcode and the parameter modes.
        # it is an integer.
        
        opcode = get_opcode(program[ip])
        
        first_instruction_value_string = str(program[ip]) 
        
        # halt
        if opcode == 99: break
            
        # addition 
        elif opcode == 1:
            while len(first_instruction_value_string) < 5: # 3 digits of parameter modes + 2 digits of opcode
                first_instruction_value_string = "0" + first_instruction_value_string
            
            # determine parameter modes
            first_instruction_value_string = first_instruction_value_string[:-2] # we dont need opcode anymore
            
            val1 = determine_values_from_parameter_mode(first_instruction_value_string[-1], 
                                                         program, 
                                                         ip,
                                                         1,
                                                         relative_base_offset)
            
            val2 = determine_values_from_parameter_mode(first_instruction_value_string[-2], 
                                                         program, 
                                                         ip,
                                                         2,
                                                         relative_base_offset)
            
            
            if int(first_instruction_value_string[-3]) == 2: # relative mode
                program[program[ip+3]+relative_base_offset] = val1 + val2
            
            else: # position mode
                program[program[ip+3]] = val1 + val2
            
            ip += opcode_parameter_map["1"] + 1

            
        # multiplication   
        elif opcode == 2:
            while len(first_instruction_value_string) < 5: # 3 digits of parameter modes + 2 digits of opcode
                first_instruction_value_string = "0" + first_instruction_value_string
                
            # determine parameter modes
            first_instruction_value_string = first_instruction_value_string[:-2] # we dont need opcode anymore
            
            val1 = determine_values_from_parameter_mode(first_instruction_value_string[-1], 
                                                         program, 
                                                         ip,
                                                         1,
                                                         relative_base_offset)
            
            val2 = determine_values_from_parameter_mode(first_instruction_value_string[-2], 
                                                         program, 
                                                         ip,
                                                         2,
                                                         relative_base_offset)
            
            
            if int(first_instruction_value_string[-3]) == 2: # relative mode
                program[program[ip+3]+relative_base_offset] = val1 * val2
            
            else: # position mode
                program[program[ip+3]] = val1 * val2
            
            
            ip += opcode_parameter_map["2"] + 1

            
        # store value
        elif opcode == 3:
            # Parameters that an instruction writes to will never be in immediate mode
            # so the input is always in position mode (.... well, this comment aged well)
            
            while len(first_instruction_value_string) < 3: # 1 digits of parameter modes + 2 digits of opcode
                first_instruction_value_string = "0" + first_instruction_value_string
                
            # determine parameter mode
            first_instruction_value_string = first_instruction_value_string[0] # we dont need opcode anymore
            
            if int(first_instruction_value_string) == 2: # relative mode
                program[program[ip+1]+relative_base_offset] = input_values.pop(0)
            
            else: # position mode
                program[program[ip+1]] = input_values.pop(0)

                
            ip += opcode_parameter_map["3"] + 1

        
        # output value
        elif opcode == 4:
            # i believe it is either possible to output the value at an address, or to output an immediate
            # value. Hence, the differentiation is needed here (even though it might be unnecessary)
            
            while len(first_instruction_value_string) < 3: # 1 digits of parameter modes + 2 digits of opcode
                first_instruction_value_string = "0" + first_instruction_value_string
                
            # determine parameter mode
            first_instruction_value_string = first_instruction_value_string[0] # we dont need opcode anymore
            
            output_value = determine_values_from_parameter_mode(first_instruction_value_string[0], 
                                                         program, 
                                                         ip,
                                                         1,
                                                         relative_base_offset)
            
            ip += opcode_parameter_map["4"] + 1
            
            # return program, ip, output_value
            print("Output: {}".format(output_value))
            
            
        # jump if true
        elif opcode == 5:
            while len(first_instruction_value_string) < 4: # 2 digits of parameter modes + 2 digits of opcode
                first_instruction_value_string = "0" + first_instruction_value_string
                
            # determine parameter modes
            first_instruction_value_string = first_instruction_value_string[:-2]
            
            condition = determine_values_from_parameter_mode(first_instruction_value_string[-1], 
                                                         program, 
                                                         ip,
                                                         1,
                                                         relative_base_offset)
            
            new_ip_position = determine_values_from_parameter_mode(first_instruction_value_string[-2], 
                                                         program, 
                                                         ip,
                                                         2,
                                                         relative_base_offset)
            
            if condition:
                ip = new_ip_position
            else:
                ip += opcode_parameter_map["5"] + 1
                
        
        # jump if false
        elif opcode == 6:
            while len(first_instruction_value_string) < 4:
                first_instruction_value_string = "0" + first_instruction_value_string
                
            # determine parameter modes
            first_instruction_value_string = first_instruction_value_string[:-2]
            
            condition = determine_values_from_parameter_mode(first_instruction_value_string[-1], 
                                                         program, 
                                                         ip,
                                                         1,
                                                         relative_base_offset)
            
            new_ip_position = determine_values_from_parameter_mode(first_instruction_value_string[-2], 
                                                         program, 
                                                         ip,
                                                         2,
                                                         relative_base_offset)
            
            
            if not condition:
                ip = new_ip_position
            else:
                ip += opcode_parameter_map["6"] + 1
        
        # less than
        elif opcode == 7:
            while len(first_instruction_value_string) < 5:
                first_instruction_value_string = "0" + first_instruction_value_string
                
            # determine parameter modes
            first_instruction_value_string = first_instruction_value_string[:-2]
            
            val1 = determine_values_from_parameter_mode(first_instruction_value_string[-1], 
                                                         program, 
                                                         ip,
                                                         1,
                                                         relative_base_offset)
            
            val2 = determine_values_from_parameter_mode(first_instruction_value_string[-2], 
                                                         program, 
                                                         ip,
                                                         2,
                                                         relative_base_offset)
            
            if val1 < val2:
                
                if int(first_instruction_value_string[-3]) == 2: # relative mode
                    program[program[ip+3]+relative_base_offset] = 1
            
                else: # position mode
                    program[program[ip+3]] = 1
                          
                
            else:
                if int(first_instruction_value_string[-3]) == 2: # relative mode
                    program[program[ip+3]+relative_base_offset] = 0
            
                else: # position mode
                    program[program[ip+3]] = 0
                
            ip += opcode_parameter_map["7"] + 1
                
        
        # equals
        elif opcode == 8:
            while len(first_instruction_value_string) < 5:
                first_instruction_value_string = "0" + first_instruction_value_string
                
            # determine parameter modes
            first_instruction_value_string = first_instruction_value_string[:-2]
            
            val1 = determine_values_from_parameter_mode(first_instruction_value_string[-1], 
                                                         program, 
                                                         ip,
                                                         1,
                                                         relative_base_offset)
            
            val2 = determine_values_from_parameter_mode(first_instruction_value_string[-2], 
                                                         program, 
                                                         ip,
                                                         2,
                                                         relative_base_offset)
            
            
            
            if val1 == val2:
                
                if int(first_instruction_value_string[-3]) == 2: # relative mode
                    program[program[ip+3]+relative_base_offset] = 1
            
                else: # position mode
                    program[program[ip+3]] = 1
                          
                
            else:
                if int(first_instruction_value_string[-3]) == 2: # relative mode
                    program[program[ip+3]+relative_base_offset] = 0
            
                else: # position mode
                    program[program[ip+3]] = 0
                
            ip += opcode_parameter_map["8"] + 1
            
            
            
        # adjust relative base
        elif opcode == 9:
            while len(first_instruction_value_string) < 3: # 1 digits of parameter modes + 2 digits of opcode
                first_instruction_value_string = "0" + first_instruction_value_string
                
            # determine parameter mode            
            value = determine_values_from_parameter_mode(first_instruction_value_string[0], 
                                                         program, 
                                                         ip,
                                                         1,
                                                         relative_base_offset)
            
            
            relative_base_offset += value
            ip += opcode_parameter_map["9"] + 1
            
            
            
        
            

    # return program, ip, output_value
    return program, output_value

In [10]:
intcode_computer(puzzle_input, [1], 0)

Output: 2932210790


([1102,
  34463338,
  34463338,
  63,
  1007,
  63,
  34463338,
  63,
  1005,
  63,
  53,
  1101,
  3,
  0,
  1000,
  109,
  988,
  209,
  12,
  9,
  1000,
  209,
  6,
  209,
  3,
  203,
  0,
  1008,
  1000,
  1,
  63,
  1005,
  63,
  65,
  1008,
  1000,
  2,
  63,
  1005,
  63,
  904,
  1008,
  1000,
  0,
  63,
  1005,
  63,
  58,
  4,
  25,
  104,
  0,
  99,
  4,
  0,
  104,
  0,
  99,
  4,
  17,
  104,
  0,
  99,
  1,
  2932210790,
  1101,
  35,
  0,
  1007,
  1102,
  30,
  1,
  1013,
  1102,
  37,
  1,
  1017,
  1101,
  23,
  0,
  1006,
  1101,
  0,
  32,
  1008,
  1102,
  1,
  29,
  1000,
  1101,
  0,
  38,
  1010,
  1101,
  0,
  24,
  1002,
  1101,
  33,
  0,
  1003,
  1101,
  1,
  0,
  1021,
  1102,
  31,
  1,
  1019,
  1101,
  27,
  0,
  1014,
  1102,
  20,
  1,
  1005,
  1101,
  0,
  0,
  1020,
  1102,
  1,
  892,
  1027,
  1101,
  895,
  0,
  1026,
  1102,
  39,
  1,
  1015,
  1102,
  1,
  370,
  1029,
  1102,
  1,
  28,
  1001,
  1102,
  34,
  1,
  1012,
  1101,
  25,
  0,
 